In [134]:
import requests
import json  ### Intentar cambiar el formato JSON por uno más específico de Python
import sys
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

## Aceso a todas las variantes de un gen a través de la API de Ensembl

In [135]:
def acceso_variantes_ensembl (identificador_gen, especie):
    """ 
    DESCRIPCIÓN DE LA FUNCIÓN"""
    
    rest_api = "https://rest.ensembl.org"
    extension_variantes = f"/overlap/id/{identificador_gen}?feature=variation;species={especie}"

    solicitud = requests.get(rest_api+extension_variantes, headers={"Content-Type" : "application/json"})


    if not solicitud.ok:
        solicitud.raise_for_status()
        sys.exit()

    variantes = solicitud.json()
    
    return variantes



## Obtención de las variantes patogénicas a partir de las variantes que hemos obtenido de la API de Ensembl

In [136]:
def obtener_variantes_patogenicas (identificador_gen, especie):
    """
    DESCRIPCIÓN DE LA FUNCIÓN
    """

    variantes = acceso_variantes_ensembl (identificador_gen, especie)
    variantes_patogenicas = []

    for var in variantes:
        if "clinical_significance" in var and "pathogenic" in var["clinical_significance"]:
            variantes_patogenicas.append(var)


    return variantes_patogenicas
            

## Obtención de las variantes benignas a partir de las variantes que hemos obtenido de la API de Ensembl

In [137]:
def obtener_variantes_benignas (identificador_gen, especie):
    """
    DESCRIPCIÓN DE LA FUNCIÓN
    """

    variantes = acceso_variantes_ensembl (identificador_gen, especie)
    variantes_benignas = []

    for var in variantes:
        if "clinical_significance" in var and "benign" in var["clinical_significance"]:
            variantes_benignas.append(var)


    return variantes_benignas
            

In [138]:
variantes_benignas = obtener_variantes_benignas ("ENST00000269305", "human")

In [140]:
for i in variantes_benignas[:5]:
    print (i)

{'id': 'rs78378222', 'clinical_significance': ['uncertain significance', 'benign', 'risk factor'], 'assembly_name': 'GRCh38', 'start': 7668434, 'seq_region_name': '17', 'strand': 1, 'feature_type': 'variation', 'source': 'dbSNP', 'alleles': ['T', 'G'], 'consequence_type': '3_prime_UTR_variant', 'end': 7668434}
{'start': 7668539, 'clinical_significance': ['benign', 'likely benign'], 'assembly_name': 'GRCh38', 'id': 'rs114831472', 'end': 7668539, 'source': 'dbSNP', 'feature_type': 'variation', 'alleles': ['G', 'A', 'C'], 'consequence_type': '3_prime_UTR_variant', 'seq_region_name': '17', 'strand': 1}
{'seq_region_name': '17', 'strand': 1, 'alleles': ['C', 'A', 'G', 'T'], 'feature_type': 'variation', 'source': 'dbSNP', 'consequence_type': '3_prime_UTR_variant', 'end': 7668783, 'id': 'rs17884306', 'clinical_significance': ['benign'], 'assembly_name': 'GRCh38', 'start': 7668783}
{'seq_region_name': '17', 'strand': 1, 'consequence_type': '3_prime_UTR_variant', 'alleles': ['G', 'A', 'T'], 'fe

In [141]:
variantes_patogenicas = obtener_variantes_patogenicas ("ENST00000269305", "human")

In [142]:
for i in variantes_patogenicas[:5]:
    print (i)

{'clinical_significance': ['pathogenic'], 'seq_region_name': '17', 'id': 'rs730882017', 'feature_type': 'variation', 'end': 7669666, 'consequence_type': 'frameshift_variant', 'assembly_name': 'GRCh38', 'source': 'dbSNP', 'start': 7669666, 'alleles': ['C', '-'], 'strand': 1}
{'start': 7669673, 'alleles': ['TTTTT', 'TTTT'], 'strand': 1, 'feature_type': 'variation', 'end': 7669677, 'consequence_type': 'frameshift_variant', 'assembly_name': 'GRCh38', 'source': 'dbSNP', 'clinical_significance': ['uncertain significance', 'pathogenic'], 'seq_region_name': '17', 'id': 'rs2150988776'}
{'strand': 1, 'alleles': ['C', 'A', 'T'], 'start': 7669691, 'source': 'dbSNP', 'assembly_name': 'GRCh38', 'consequence_type': 'splice_acceptor_variant', 'feature_type': 'variation', 'end': 7669691, 'id': 'rs876658982', 'seq_region_name': '17', 'clinical_significance': ['likely pathogenic', 'pathogenic']}
{'feature_type': 'variation', 'end': 7669692, 'consequence_type': 'splice_acceptor_variant', 'assembly_name': 

## Obtención de la localización de las variantes

In [159]:
def obtener_localizacion_variantes (variantes):

    diccionario_posiciones_variantes = {}

    for var in variantes:
        inicio = var["start"]
        final = var["end"]
        longitud = final - (inicio - 1)
        diccionario_posiciones_variantes[var["id"]] = [inicio, final, longitud]
    
    return diccionario_posiciones_variantes
        

## Obtención de los exones

In [148]:
def obtener_exones_gen (identificador_transcrito, especie):
    
    rest_api = "https://rest.ensembl.org"
    extension_exones = f"/lookup/id/{identificador_transcrito}?expand=1&species={especie}"

    solicitud = requests.get(rest_api+extension_exones, headers={"Content-Type" : "application/json"})

    if not solicitud.ok:
        solicitud.raise_for_status()
        sys.exit()

    exones = solicitud.json()

    return exones["Exon"]

    

In [149]:
exones = obtener_exones_gen ("ENST00000269305", "human")

for i in exones:
    print(i)

{'id': 'ENSE00003753508', 'strand': -1, 'seq_region_name': '17', 'species': 'homo_sapiens', 'version': 2, 'object_type': 'Exon', 'end': 7687490, 'start': 7687377, 'assembly_name': 'GRCh38', 'db_type': 'core'}
{'strand': -1, 'id': 'ENSE00004023728', 'object_type': 'Exon', 'seq_region_name': '17', 'species': 'homo_sapiens', 'version': 1, 'start': 7676521, 'db_type': 'core', 'assembly_name': 'GRCh38', 'end': 7676622}
{'strand': -1, 'id': 'ENSE00002419584', 'start': 7676382, 'assembly_name': 'GRCh38', 'db_type': 'core', 'end': 7676403, 'object_type': 'Exon', 'version': 1, 'seq_region_name': '17', 'species': 'homo_sapiens'}
{'id': 'ENSE00003625790', 'strand': -1, 'version': 1, 'seq_region_name': '17', 'species': 'homo_sapiens', 'object_type': 'Exon', 'end': 7676272, 'assembly_name': 'GRCh38', 'start': 7675994, 'db_type': 'core'}
{'id': 'ENSE00003518480', 'strand': -1, 'start': 7675053, 'assembly_name': 'GRCh38', 'db_type': 'core', 'end': 7675236, 'version': 1, 'seq_region_name': '17', 'spec

## Obtención de la secuencia de los exones en formato fasta

In [413]:
def obtener_secuencia_exones_fasta (identificador_transcrito, especie, nombre_archivo_fasta):
    exones = obtener_exones_gen (identificador_transcrito, especie)

    lista_secuencias = []

    contador = 1

    for ex in exones:
        identificador_exon = ex["id"]

        rest_api = "https://rest.ensembl.org"
        extension_secuencia_exon = f"/sequence/id/{identificador_exon}?content-type=text/x-fasta"

        solicitud2 = requests.get(rest_api+extension_secuencia_exon, headers={ "Content-Type" : "application/json"})

        if not solicitud2.ok:
            solicitud2.raise_for_status()
            sys.exit()

        exon = solicitud2.json()
        
        if ex["strand"] == -1:
            secuencia_fasta = SeqRecord (Seq(exon["seq"]).complement())
            
        else:
            secuencia_fasta = SeqRecord (Seq(exon["seq"]))
            
        secuencia_fasta.id = exon["id"]
        secuencia_fasta.name = exon["id"] + "exon" + str(contador)
        secuencia_fasta.description = exon["desc"]  + " exon " + str(contador)
        lista_secuencias.append(secuencia_fasta)

        contador +=1

    SeqIO.write(lista_secuencias, nombre_archivo_fasta, "fasta")

    print ("Se han guardado las secuencias correctamente")
    
    return exones


In [414]:
exones_fasta = obtener_secuencia_exones_fasta ("ENST00000269305", "human", "data/holu.fasta")


Se han guardado las secuencias correctamente


In [415]:
print (exones_fasta[1])

{'db_type': 'core', 'id': 'ENSE00004023728', 'version': 1, 'species': 'homo_sapiens', 'object_type': 'Exon', 'seq_region_name': '17', 'assembly_name': 'GRCh38', 'end': 7676622, 'strand': -1, 'start': 7676521}


## Obtención de la localización de los exones

In [257]:
def obtener_localizacion_exones (exones):
    
    diccionario_posiciones_exones = {}

    for exon in exones["Exon"]:
        inicio = exon["start"]
        final = exon["end"]
        longitud = final - inicio
        diccionario_posiciones_exones[exon["id"]] = [inicio, final, longitud]
    
    return diccionario_posiciones_exones
        
    

## Obtención de las variantes que se encuentran dentro de los exones

In [312]:
secuencia = SeqIO.parse ("data/holu.fasta", "fasta")


In [313]:
secuencia1 = next(secuencia)



In [314]:
sec = "exon 1"

In [315]:
if secuencia1.description.find ("exon1") != -1:
    hola = 1
    print (hola)

In [316]:
print (secuencia1)
print (secuencia1.seq)

ID: ENSE00003753508
Name: ENSE00003753508
Description: ENSE00003753508 chromosome:GRCh38:17:7687377:7687490:-1 exon 1
Number of features: 0
Seq('GAGTTTTCAGATCTCGGTGGCAGGTCCCTCGTCCATCGACGACCCGAGGCCCCT...ACC')
GAGTTTTCAGATCTCGGTGGCAGGTCCCTCGTCCATCGACGACCCGAGGCCCCTGTGAAACGCAAGCCCGACCCTCGCACGAAAGGTGCTGCCACTGTGCGAAGGGACCTAACC


In [317]:
secuencia2 = next(secuencia)

In [318]:
print (secuencia2)
print (secuencia2.seq)

ID: ENSE00004023728
Name: ENSE00004023728
Description: ENSE00004023728 chromosome:GRCh38:17:7676521:7676622:-1 exon 2
Number of features: 0
Seq('GTCGGTCTGACGGAAGGCCCAGTGACGGTACCTCCTCGGCGTCAGTCTAGGATC...TGA')
GTCGGTCTGACGGAAGGCCCAGTGACGGTACCTCCTCGGCGTCAGTCTAGGATCGCAGCTCGGGGGAGACTCAGTCCTTTGTAAAAGTCTGGATACCTTTGA


In [442]:
for i in SeqIO.parse ("data/holu.fasta", "fasta"):
    if i.id == "ENSE00003625790":
        secuencia3 = i
        
print (secuencia3.seq)

AGGGGGAACGGCAGGGTTCGTTACCTACTAAACTACGACAGGGGCCTGCTATAACTTGTTACCAAGTGACTTCTGGGTCCAGGTCTACTTCGAGGGTCTTACGGTCTCCGACGAGGGGGGCACCGGGGACGTGGTCGTCGAGGATGTGGCCGCCGGGGACGTGGTCGGGGGAGGACCGGGGACAGTAGAAGACAGGGAAGGGTCTTTTGGATGGTCCCGTCGATGCCAAAGGCAGACCCGAAGAACGTAAGACCCTGTCGGTTCAGACACTGAACGTGC


## Obtener un diccionario con todas las variantes de un determinado exón 

In [443]:
def obtener_variantes_exon (variantes, identificador_exon, exones):
    for ex in exones:
        if ex["id"] == identificador_exon:
            inicio_exon = ex["start"]
            final_exon = ex["end"]

            diccionario_variantes_exon = {}

            for var in variantes:
                inicio_variante = var["start"]

                if inicio_exon <= inicio_variante <= final_exon:
                    diccionario_variantes_exon[var["id"]] = var

    return diccionario_variantes_exon
    

In [444]:
diccionario_variantes_exon = obtener_variantes_exon (variantes_patogenicas, "ENSE00003625790", exones)
print (diccionario_variantes_exon)

{'rs55863639': {'id': 'rs55863639', 'clinical_significance': ['not provided', 'pathogenic'], 'seq_region_name': '17', 'start': 7675994, 'alleles': ['C', 'A', 'G', 'T'], 'strand': 1, 'assembly_name': 'GRCh38', 'source': 'dbSNP', 'feature_type': 'variation', 'end': 7675994, 'consequence_type': 'splice_region_variant'}, 'rs786201057': {'id': 'rs786201057', 'clinical_significance': ['uncertain significance', 'likely pathogenic', 'pathogenic'], 'seq_region_name': '17', 'assembly_name': 'GRCh38', 'source': 'dbSNP', 'feature_type': 'variation', 'consequence_type': 'missense_variant', 'end': 7675995, 'alleles': ['G', 'A', 'C', 'T'], 'start': 7675995, 'strand': 1}, 'rs2073448940': {'source': 'dbSNP', 'assembly_name': 'GRCh38', 'end': 7676007, 'feature_type': 'variation', 'consequence_type': 'splice_donor_variant', 'strand': 1, 'start': 7675996, 'alleles': ['TGCAAGTCACAG', 'TGCAAGTCACAGTTGCAAGTCACAG'], 'id': 'rs2073448940', 'seq_region_name': '17', 'clinical_significance': ['pathogenic']}, 'rs15

In [326]:
for i in diccionario_variantes_exon.values():
    print (len(i["alleles"][0]), len(i["alleles"][1]))

4 3
3 2
1 1
5 6


## Obtención de la secuencia de las variantes, tanto patogénicas, como benignas

Como al utilizar la API de Ensembl para obtener las variantes no obtenemos la secuencia completa del gen con estas variaciones, sino que solo obtenemos los alelos que han sufrido la variación lo que vamos a hacer es lo siguiente:

1. A partir del diccionario de las variantes que corresponden a un determinado exón, vamos a identificar el tipo de variación que ha sufrido la secuencia, una inserción (el alelo de referencia es más corto que el alternativo), una delección (el alelo de referencia tiene una mayor longitud que el alternativo) y un reemplazo (el alelo de referencia y el alternativo presentan la misma longitud). 
   
3. Realizamos la modificación de la secuencia del exón con el alelo mutado.  

In [424]:
def obtener_secuencia_variantes_exon (diccionario_variantes_exon, exones, exon_fasta):

    for ex in exones:
        if ex["id"] == exon_fasta.id:
            inicio_exon = ex["start"]
            final_exon = ex["end"]
            sec_exon_fasta = exon_fasta.seq
            
            secs =[]
            
            for var in diccionario_variantes_exon.values():
                if len(var["alleles"][0]) > len(var["alleles"][1]):
                    longitud_variante = len(var["alleles"][0])
                    inicio_variante = final_exon - var["start"] - len(var["alleles"][0])
                    secuencia_variante = sec_exon_fasta[:inicio_variante+1]+var["alleles"][1]+sec_exon_fasta[inicio_variante+longitud_variante+1:]
                    secs.append ([var["id"],secuencia_variante])

                elif len(var["alleles"][0]) < len(var["alleles"][1]):
                    longitud_variante = len(var["alleles"][0])
                    inicio_variante = final_exon - var["start"] - len(var["alleles"][0])
                    secuencia_variante = sec_exon_fasta[:inicio_variante+1]+var["alleles"][1]+sec_exon_fasta[inicio_variante+longitud_variante+1:]
                    secs.append ([var["id"],secuencia_variante])

                else:
                    longitud_variante = len(var["alleles"][0])
                    inicio_variante = final_exon - var["start"]
                    secuencia_variante = sec_exon_fasta[:inicio_variante]+var["alleles"][1]+sec_exon_fasta[inicio_variante+longitud_variante:]
                    secs.append ([var["id"],secuencia_variante])
                    
                    

    return secs
        

In [445]:
ab = obtener_secuencia_variantes_exon (diccionario_variantes_exon, exones, secuencia3)
print (secuencia3.seq)
print (ab[-1])

AGGGGGAACGGCAGGGTTCGTTACCTACTAAACTACGACAGGGGCCTGCTATAACTTGTTACCAAGTGACTTCTGGGTCCAGGTCTACTTCGAGGGTCTTACGGTCTCCGACGAGGGGGGCACCGGGGACGTGGTCGTCGAGGATGTGGCCGCCGGGGACGTGGTCGGGGGAGGACCGGGGACAGTAGAAGACAGGGAAGGGTCTTTTGGATGGTCCCGTCGATGCCAAAGGCAGACCCGAAGAACGTAAGACCCTGTCGGTTCAGACACTGAACGTGC
['rs1131691041', Seq('AAGGGGGAACGGCAGGGTTCGTTACCTACTAAACTACGACAGGGGCCTGCTATA...TGC')]


## Obtención de la secuencia de las variantes patogénicas en formato fasta

Como las variantes que hemos obtenido en un primer paso de obtención de variantes sabemos que corresponden a humanos, no necesitamso especificar la especie en este caso.

Vamos a crear un objeto tipo SeqRecord con los datos de la variante y lo vamos a almacenar en un archivo de tipo fasta

In [446]:
def obtener_secuencia_variantes_fasta (diccionario_variantes_exon, secuencia_variantes, nombre_archivo_fasta, exon_fasta):
    lista_secuencias = []

    for var in diccionario_variantes_exon.values():
        for seq in secuencia_variantes:
            if var["id"] == seq[0]:
                secuencia_fasta = SeqRecord (Seq(seq[1]))
                secuencia_fasta.id = var["id"]
                secuencia_fasta.name = var["id"]
                secuencia_fasta.description = str(var["clinical_significance"]) + str (exon_fasta.id) + str (exon_fasta.description)
                lista_secuencias.append (secuencia_fasta)

    SeqIO.write(lista_secuencias, nombre_archivo_fasta, "fasta")

    print ("Las secuencias de las variantes se han guardado en formato fasta")
                
        
    
    

In [447]:
obtener_secuencia_variantes_fasta (diccionario_variantes_exon, ab, "data/secuencia_variantes.fasta", secuencia3)

Las secuencias de las variantes se han guardado en formato fasta
